In [26]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv("../datasets/titanic/train.csv")

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [42]:
selected_cols = ['Pclass', 'Survived', 'Sex', 'Age', 'SibSp', 'Parch']

In [43]:
data = df.iloc?

In [ ]:
data = df.iloc

In [44]:
data = df.loc[:, selected_cols]

In [45]:
le = LabelEncoder()

In [46]:
data["Sex"] = le.fit_transform(data.Sex)

In [47]:
data.head()

,Pclass,Survived,Sex,Age,SibSp,Parch
0,3,0,1,22.0,1,0
1,1,1,0,38.0,1,0
2,3,1,0,26.0,0,0
3,1,1,0,35.0,1,0
4,3,0,1,35.0,0,0


In [48]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
Pclass      891 non-null int64
Survived    891 non-null int64
Sex         891 non-null int64
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
dtypes: float64(1), int64(5)
memory usage: 41.8 KB


In [51]:
data.Age.fillna(np.mean(data.Age), inplace=True)

In [52]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
Pclass      891 non-null int64
Survived    891 non-null int64
Sex         891 non-null int64
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
dtypes: float64(1), int64(5)
memory usage: 41.8 KB


In [53]:
le.classes_

array(['female', 'male'], dtype=object)

In [54]:
def entropy(column):
    vals, counts = np.unique(column, return_counts=True)
    
    acc = 0
    
    for count in counts:
        p = count / len(column)
        acc += p*np.log2(p)
        
    return -acc

In [55]:
entropy(data.Survived)

0.9607079018756469

In [62]:
def info_gain(X, y, label):
    pivot = np.mean(X[label])
    
    y_left = y[X[label] < pivot]
    y_right = y[X[label] >= pivot]
    
    if (len(y_left) == 0) or (len(y_right) == 0):
        return -1000
    
    p_left = (len(y_left)/len(y))
    p_right = (len(y_right)/len(y))
    
    return entropy(y) - p_left*entropy(y_left) - p_right*entropy(y_right)

In [63]:
X = data.loc[:, ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch']]
y = data['Survived']

In [64]:
for label in X.columns:
    print(label, info_gain(X, y, label))

Pclass 0.07579362743608165
Sex 0.2176601066606143
Age 0.001158644038169343
SibSp 0.009584541813400127
Parch 0.015380754493137666


In [65]:
np.mean(data.Sex)

0.6475869809203143

In [66]:
class Node:
    def __init__(self, label=None, value=None, result=None):
        self.label = label
        self.value = value
        self.result = result

In [170]:
class DecisionTree:
    def __init__(self, max_depth=5):
        self.max_depth = max_depth
        
    def fit(self, X, y):
#         self.X = X
#         self.y = y
        
        self.root = self.generate(X, y, self.max_depth)
    
    def generate(self, X, y, depth):
        
        if depth == 1:
            return Node(result=np.mean(y))
        
        gains = []
        for label in X.columns:
            gain = info_gain(X, y, label)
            gains.append((gain, label))
        
        selected_label = max(gains)[1]
        pivot = np.mean(X[selected_label])
        
        left_X = X[X[selected_label] < pivot]
        right_X = X[X[selected_label] >= pivot]
        left_y = y[X[selected_label] < pivot]
        right_y = y[X[selected_label] >= pivot]
        
        if (len(left_y) == 0) or (len(right_y) == 0):
            return Node(result=np.mean(y))
        
        node = Node(selected_label, pivot)
        node.left = self.generate(left_X, left_y, depth-1)
        node.right = self.generate(right_X, right_y, depth-1)
        
        return node
    
    def display(self, node, indent=""):
        
        if(node.label == None):
            if node.result < .5:
                print(indent, "Died")
            else:
                print(indent, "Survived")
            return
        
        self.display(node.left, indent+"\t")
        self.display(node.right, indent+"\t")
        
    def predict_one(self, item, node):
        if node.label == None:
            return node.result
        pivot = node.value
        if(item[node.label] < pivot):
            return self.predict_one(item, node.left)
        else:
            return self.predict_one(item, node.right)
    
    def predict(self, X):
        y = []
        for row in X.iterrows():
            res = self.predict_one(row[1], self.root)
            y.append(int(res >= .5))
            
        return np.array(y)
    
    def score(self, X, y):
        yp = self.predict(X)
        return sum(y.values == yp) / len(y)
        

In [171]:
data.head()

,Pclass,Survived,Sex,Age,SibSp,Parch
0,3,0,1,22.0,1,0
1,1,1,0,38.0,1,0
2,3,1,0,26.0,0,0
3,1,1,0,35.0,1,0
4,3,0,1,35.0,0,0


In [193]:
from sklearn.model_selection import train_test_split

In [196]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

In [227]:
model = DecisionTree(max_depth=15)

In [228]:
model.fit(X_train, y_train)

In [229]:
# model.display(model.root)

In [230]:
# model.predict(X.loc[:10])

In [231]:
# y[:10].values

In [232]:
for row in X.loc[:4].iterrows():
    print(type(row[1]))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [233]:
model.score(X_test, y_test)

0.7796610169491526